In [1]:
import pandas as pd
import matplotlib as plt
import math
import numpy as np
import geopandas as gpd
import os

In [2]:
pd.set_option('display.float_format', '{:.2f}'.format)  # Muestra 2 decimales

In [3]:
CPV_10 = pd.read_csv('Datos/resageburb_21_2010_csv/resultados_ageb_urbana_21_cpv2010/conjunto_de_datos/resultados_ageb_urbana_21_cpv2010.csv', delimiter=',')

In [4]:
CPV_10.columns = [col.upper() for col in CPV_10.columns]
CPV_10 = CPV_10.rename(columns={'PRESOE05': 'PRESOE',
                                'PRESOE05_F':'PRESOE_F',
                                'PRESOE05_M':'PRESOE_M',
                                'PRES2005_M':'PRES_M',
                                'PRES2005_F':'PRES_F',
                                'PRES2005':'PRES',
                                'PCON_LIM':'PCON_DISC',
                                'PCLIM_MOT':'PCDISC_MOT',
                                'PCLIM_VIS':'PCDISC_VIS',
                                'PCLIM_LENG':'PCDISC_LENG',
                                'PCLIM_AUD':'PCDISC_AUD',
                                'PCLIM_MOT2':'PCDISC_MOT2',
                                'PCLIM_MEN':'PCDISC_MEN',
                                'PCLIM_MEN2':'PCLIM_PMEN',
                                'PSIN_LIM':'PSIND_LIM',
                                'PNCATOLICA':'PRO_CRIEVA'})
mask = CPV_10['MUN'] == 114
CPV_10 = CPV_10[mask]
CPV_10

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBMAS,...,VPH_SNBIEN,VPH_RADIO,VPH_TV,VPH_REFRI,VPH_LAVAD,VPH_AUTOM,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER
23872,21,Puebla,114,Puebla,0,Total del municipio,0000,0,1539819,734352,...,2603,353839,382681,333558,275198,172378,158646,235674,292074,118384
23873,21,Puebla,114,Puebla,1,Total de la localidad urbana,0000,0,1434062,682505,...,1545,335000,360729,321000,266259,166406,156265,229857,281036,117133
23874,21,Puebla,114,Puebla,1,Total AGEB urbana,0126,0,4819,2275,...,4,1136,1232,1108,913,619,601,845,927,491
23875,21,Puebla,114,Puebla,1,Heróica Puebla de Zaragoza,0126,1,609,280,...,*,141,150,121,94,45,35,77,101,29
23876,21,Puebla,114,Puebla,1,Heróica Puebla de Zaragoza,0126,3,139,61,...,0,31,36,33,24,15,16,28,24,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40699,21,Puebla,114,Puebla,488,Galaxia la Calera,6727,28,0,0,...,0,0,0,0,0,0,0,0,0,0
40700,21,Puebla,114,Puebla,488,Galaxia la Calera,6727,29,0,0,...,0,0,0,0,0,0,0,0,0,0
40701,21,Puebla,114,Puebla,488,Galaxia la Calera,6727,30,0,0,...,0,0,0,0,0,0,0,0,0,0
40702,21,Puebla,114,Puebla,488,Galaxia la Calera,6727,31,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
CPV_10=CPV_10[CPV_10['MZA']==0]
CPV_10=CPV_10[CPV_10['AGEB']!='0000']
AGEB_CPV=CPV_10['AGEB'].unique()
CPV_10.sort_values('AGEB')
agebs_cpv = set(CPV_10['AGEB'].unique())
CPV_10=CPV_10.set_index('AGEB')
CPV_10=CPV_10.sort_index()
CPV_10

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,MZA,POBTOT,POBMAS,POBFEM,...,VPH_SNBIEN,VPH_RADIO,VPH_TV,VPH_REFRI,VPH_LAVAD,VPH_AUTOM,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER
AGEB,,,,,,,,,,,,,,,,,,,,,
0126,21,Puebla,114,Puebla,1,Total AGEB urbana,0,4819,2275,2544,...,4,1136,1232,1108,913,619,601,845,927,491
0130,21,Puebla,114,Puebla,1,Total AGEB urbana,0,6423,3057,3366,...,8,1538,1680,1500,1157,643,635,908,1316,422
0198,21,Puebla,114,Puebla,1,Total AGEB urbana,0,5256,2458,2798,...,3,1276,1373,1236,969,518,570,875,1014,415
0338,21,Puebla,114,Puebla,1,Total AGEB urbana,0,3406,1632,1774,...,6,768,845,723,617,310,269,541,593,168
0361,21,Puebla,114,Puebla,1,Total AGEB urbana,0,1789,845,944,...,0,429,457,446,392,308,321,385,386,282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,21,Puebla,114,Puebla,1,Total AGEB urbana,0,18,8,10,...,0,3,4,3,*,0,*,0,*,0
6680,21,Puebla,114,Puebla,1,Total AGEB urbana,0,935,479,456,...,25,155,185,40,40,23,6,6,141,*
6695,21,Puebla,114,Puebla,359,Total AGEB urbana,0,145,74,71,...,*,30,33,16,10,4,3,*,26,*


In [6]:
ruta = '/home/javier-rica-o/Escritorio/Ciencia de datos/Proyecto FInal/Datos/geostadistico_2010'

# Lista para guardar cada GeoDataFrame
geodfs = []

for archivo in os.listdir(ruta):
    if archivo.endswith('A.shp') and not archivo.endswith("IA.shp") and not archivo.endswith("CA.shp"):
        gdf = gpd.read_file(os.path.join(ruta, archivo))
        geodfs.append(gdf)

# Unir todos los shapefiles en uno solo
ageb_2010 = gpd.GeoDataFrame(pd.concat(geodfs, ignore_index=True))

# Guardar shapefile unido (opcional)
#agebs_puebla.to_file('Datos/agebs_puebla_2010_unido.shp')


In [7]:
ageb_2010['AGEB'] = ageb_2010['CVEGEO'].astype(str).str[-4:]
agebs_2010 = set(ageb_2010['AGEB'].unique())
ageb_2010=ageb_2010.set_index('AGEB')
ageb_2010=ageb_2010.sort_index()

In [8]:
ageb_2010

,CODIGO,CVEGEO,GEOGRAFICO,FECHAACT,GEOMETRIA,INSTITUCIO,geometry
AGEB,,,,,,,
0126,LA_6052,2111400010126,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2898224.145 790876.493, 2898177.799 ..."
0130,LA_6052,2111400010130,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2899295.428 790736.219, 2899196.837 ..."
0198,LA_6052,2111400010198,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2894649.682 791049.840, 2894598.676 ..."
0338,LA_6052,2111400010338,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2903500.313 789608.302, 2903450.076 ..."
0361,LA_6052,2111400010361,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896010.210 788743.321, 2895949.550 ..."
...,...,...,...,...,...,...,...
6676,LA_6052,2111400016676,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896630.713 779057.627, 2896652.213 ..."
6680,LA_6052,2111400016680,ÁREA GEOESTADÍSTICA BÁSICA,04/2010,ÁREA,INEGI,"POLYGON ((2893816.679 778660.123, 2893817.976 ..."
6695,LA_6052,2111403596695,ÁREA GEOESTADÍSTICA BÁSICA,05/2010,ÁREA,INEGI,"POLYGON ((2902665.816 781291.165, 2902472.591 ..."


In [9]:

faltantes = agebs_cpv - agebs_2010

print(faltantes)


set()


In [10]:
AGEB_2010 = ageb_2010.join(CPV_10, how='left')
AGEB_2010

,CODIGO,CVEGEO,GEOGRAFICO,FECHAACT,GEOMETRIA,INSTITUCIO,geometry,ENTIDAD,NOM_ENT,MUN,...,VPH_SNBIEN,VPH_RADIO,VPH_TV,VPH_REFRI,VPH_LAVAD,VPH_AUTOM,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER
AGEB,,,,,,,,,,,,,,,,,,,,,
0126,LA_6052,2111400010126,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2898224.145 790876.493, 2898177.799 ...",21,Puebla,114,...,4,1136,1232,1108,913,619,601,845,927,491
0130,LA_6052,2111400010130,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2899295.428 790736.219, 2899196.837 ...",21,Puebla,114,...,8,1538,1680,1500,1157,643,635,908,1316,422
0198,LA_6052,2111400010198,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2894649.682 791049.840, 2894598.676 ...",21,Puebla,114,...,3,1276,1373,1236,969,518,570,875,1014,415
0338,LA_6052,2111400010338,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2903500.313 789608.302, 2903450.076 ...",21,Puebla,114,...,6,768,845,723,617,310,269,541,593,168
0361,LA_6052,2111400010361,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896010.210 788743.321, 2895949.550 ...",21,Puebla,114,...,0,429,457,446,392,308,321,385,386,282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,LA_6052,2111400016676,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896630.713 779057.627, 2896652.213 ...",21,Puebla,114,...,0,3,4,3,*,0,*,0,*,0
6680,LA_6052,2111400016680,ÁREA GEOESTADÍSTICA BÁSICA,04/2010,ÁREA,INEGI,"POLYGON ((2893816.679 778660.123, 2893817.976 ...",21,Puebla,114,...,25,155,185,40,40,23,6,6,141,*
6695,LA_6052,2111403596695,ÁREA GEOESTADÍSTICA BÁSICA,05/2010,ÁREA,INEGI,"POLYGON ((2902665.816 781291.165, 2902472.591 ...",21,Puebla,114,...,*,30,33,16,10,4,3,*,26,*


In [11]:
DEUNE_10 = pd.read_csv('Datos/denue_21_2010_csv/DenueCSV21.csv', delimiter=',',encoding='latin1')
DEUNE_10

,Nombre de la unidad económica,Razón social,Código de la clase de actividad,Nombre de la clase de actividad,Personal ocupado (estrato),"Calle, avenida, andador, carretera, manzana u otro",Número exterior o km,"Edificio, piso o nivel",Número o letra interior,"Colonia, fraccionamiento, unidad habitacional o barrio",...,Municipio,Localidad,Área geoestadística básica,Manzana,Número de teléfono,Correo electrónico,Sitio en Internet,Tipo de unidad económica,Latitud,Longitud
0,URBANIZACIONES ACROPOLIS,URBANIZACIONES ACROPOLIS¶ÿ SA DE CV,237312,"CONSTRUCCION DE CARRETERAS, PUENTES Y SIMILARES",11 A 30 PERSONAS,NACIONAL,SN,NaN,NaN,SANTA ISABEL TEPETZALA,...,ACAJETE,Acajete,0211,51,2232726646,vacropolis@prodigy.com,NaN,FIJO,19.13,-97.94
1,PALETERIA,NaN,311520,ELABORACION DE HELADOS Y PALETAS,6 A 10 PERSONAS,3 PONIENTE,SN,NaN,NaN,BARRIO DE JESUS ACAJETE,...,ACAJETE,Acajete,0194,9,0,NaN,NaN,FIJO,19.11,-97.95
2,PELETERIA KRISTAL,NaN,311520,ELABORACION DE HELADOS Y PALETAS,0 A 5 PERSONAS,7 PONIENTE,306,NaN,NaN,BARRIO LA VIRGEN ACAJETE,...,ACAJETE,Acajete,0194,1,0,NaN,NaN,FIJO,19.10,-97.95
3,EXPENDIO DE PAN,NaN,311812,PANIFICACION TRADICIONAL,0 A 5 PERSONAS,7 ORIENTE,707,NaN,NaN,BARRIO LA VIRGEN,...,ACAJETE,Acajete,018A,1,0,NaN,NaN,FIJO,19.11,-97.95
4,PANADERIA,NaN,311812,PANIFICACION TRADICIONAL,0 A 5 PERSONAS,7 ORIENTE,SN,NaN,NaN,BARRIO LA VIRGEN,...,ACAJETE,Acajete,018A,1,2230,NaN,NaN,FIJO,19.11,-97.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252445,CAPILLA,CAPILLA SI NOMBRE,813210,ASOCIACIONES Y ORGANIZACIONES RELIGIOSAS,0 A 5 PERSONAS,SIN NOMBRE,SN,NaN,NaN,BARRIO AHUATEPEC,...,ZOQUITLÁN,Xitlama,0119,18,2360,NaN,NaN,FIJO,18.31,-97.02
252446,ASOCIACION DE PADRES DE FAMILIA,ASOCIACION DE PADRES DE FAMILIA,813230,ASOCIACIONES Y ORGANIZACIONES CIVILES,0 A 5 PERSONAS,SIN NOMBRE,SN,NaN,NaN,BARRIO 6,...,ZOQUITLÁN,Xitlama,0119,22,2360,NaN,NaN,FIJO,18.30,-97.02
252447,ASOCIACIOPN DE PADRES DE FAMILIA,ASOCIACION DE PADRES DE FAMILIA,813230,ASOCIACIONES Y ORGANIZACIONES CIVILES,0 A 5 PERSONAS,SIN NOMBRE,SN,NaN,NaN,BARRIO 7,...,ZOQUITLÁN,Xitlama,0119,12,2360,NaN,NaN,FIJO,18.30,-97.02
252448,ADMINISTRACION PUBLICA,NaN,931210,ADMINISTRACION PUBLICA EN GENERAL,6 A 10 PERSONAS,SIN NOMBRE,SN,NaN,NaN,BARRIO AHUATEPEC,...,ZOQUITLÁN,Xitlama,0119,18,2360,NaN,NaN,FIJO,18.31,-97.02


In [12]:
DEUNE_10 = DEUNE_10.rename(columns={
    'Área geoestadística básica': 'ageb',
    'Código de la clase de actividad': 'codigo_act',
    'Personal ocupado (estrato)': 'per_ocu'
})
DEUNE_10=DEUNE_10[['ageb','codigo_act','per_ocu']]
DEUNE_10

,ageb,codigo_act,per_ocu
0,0211,237312,11 A 30 PERSONAS
1,0194,311520,6 A 10 PERSONAS
2,0194,311520,0 A 5 PERSONAS
3,018A,311812,0 A 5 PERSONAS
4,018A,311812,0 A 5 PERSONAS
...,...,...,...
252445,0119,813210,0 A 5 PERSONAS
252446,0119,813230,0 A 5 PERSONAS
252447,0119,813230,0 A 5 PERSONAS
252448,0119,931210,6 A 10 PERSONAS


In [13]:
agebs_deune = set(DEUNE_10['ageb'].unique())
faltantes = agebs_cpv - agebs_deune
print(faltantes)

{'6604', '6642', '6661', '6591', '6676', '6553', '6341', '6712', '6360', '6515', '5150', '6680', '6587', '6498', '2885', '6695', '3169', '6727'}


In [14]:
DEUNE_10 = DEUNE_10[DEUNE_10['ageb'].isin(agebs_cpv)]
DEUNE_10 = DEUNE_10.sort_values('ageb')
DEUNE_10

,ageb,codigo_act,per_ocu
43513,0126,311812,0 A 5 PERSONAS
130905,0126,721112,0 A 5 PERSONAS
130901,0126,721112,0 A 5 PERSONAS
130875,0126,721112,6 A 10 PERSONAS
130871,0126,721112,0 A 5 PERSONAS
...,...,...,...
87463,6623,461110,0 A 5 PERSONAS
149066,6623,813110,0 A 5 PERSONAS
90791,6623,461110,0 A 5 PERSONAS
91565,6638,461110,0 A 5 PERSONAS


In [15]:
DEUNE_10=DEUNE_10[DEUNE_10['per_ocu']!='NO ESPECIFICADO']
DEUNE_10['per_ocu'].unique()

array(['0 A 5 PERSONAS', '6 A 10 PERSONAS', '31 A 50 PERSONAS',
       '51 A 100 PERSONAS', '11 A 30 PERSONAS', '251 Y MAS PERSONAS',
       '101 A 250 PERSONAS'], dtype=object)

In [16]:
reemplazos = {
    '0 A 5 PERSONAS': 2,
    '6 A 10 PERSONAS': 8,
    '11 A 30 PERSONAS': 20,
    '31 A 50 PERSONAS': 40,
    '51 A 100 PERSONAS': 75,
    '101 A 250 PERSONAS': 175,
    '251 Y MAS PERSONAS': 300
}
DEUNE_10['per_ocu'] = DEUNE_10['per_ocu'].replace(reemplazos)
DEUNE_10

/tmp/ipykernel_7638/3927186934.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DEUNE_10['per_ocu'] = DEUNE_10['per_ocu'].replace(reemplazos)


,ageb,codigo_act,per_ocu
43513,0126,311812,2
130905,0126,721112,2
130901,0126,721112,2
130875,0126,721112,8
130871,0126,721112,2
...,...,...,...
87463,6623,461110,2
149066,6623,813110,2
90791,6623,461110,2
91565,6638,461110,2


In [17]:
# Convertir los valores de la columna 'per_ocu' a cadenas y extraer los primeros dos dígitos
DEUNE_10['primeros_dos_digitos'] = DEUNE_10['codigo_act'].astype(str).str[:2]

# Crear una columna para cada posible valor de los primeros dos dígitos
# Esto va a crear columnas para cada valor único de los primeros dos dígitos
DEUNE_10_wide_1 = DEUNE_10.pivot_table(index='ageb', columns='primeros_dos_digitos', values='codigo_act', aggfunc='count')
DEUNE_10_wide_2 = DEUNE_10.pivot_table(index='ageb', columns='primeros_dos_digitos', values='per_ocu', aggfunc='mean')

DEUNE_10_wide_1['31-33'] = DEUNE_10_wide_1[['31', '32', '33']].sum(axis=1)
DEUNE_10_wide_1 = DEUNE_10_wide_1.drop(columns=['31', '32', '33'])

DEUNE_10_wide_2['31-33'] = DEUNE_10_wide_2[['31', '32', '33']].mean(axis=1)
DEUNE_10_wide_2 = DEUNE_10_wide_2.drop(columns=['31', '32', '33'])

DEUNE_10_wide_1['48-49'] = DEUNE_10_wide_1[['48', '49']].sum(axis=1)
DEUNE_10_wide_1 = DEUNE_10_wide_1.drop(columns=['48', '49'])

DEUNE_10_wide_2['48-49'] = DEUNE_10_wide_2[['48', '49',]].mean(axis=1)
DEUNE_10_wide_2 = DEUNE_10_wide_2.drop(columns=['48', '49'])

# Renombrar las columnas agregando un prefijo "grupo_"
DEUNE_10_wide_1.columns = [f"grupo_{col}" for col in DEUNE_10_wide_1.columns]
DEUNE_10_wide_2.columns = [f"per_ocu_{col}" for col in DEUNE_10_wide_2.columns]

# Unir las columnas de vuelta al DataFrame original si lo necesitas
DEUNE_10_final = pd.concat([DEUNE_10_wide_1,DEUNE_10_wide_2], axis=1)

# Ver el resultado
DEUNE_10_final=DEUNE_10_final.fillna(0)
DEUNE_10_final

/tmp/ipykernel_7638/2449494598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DEUNE_10['primeros_dos_digitos'] = DEUNE_10['codigo_act'].astype(str).str[:2]


,grupo_11,grupo_21,grupo_22,grupo_23,grupo_43,grupo_46,grupo_51,grupo_52,grupo_53,grupo_54,...,per_ocu_56,per_ocu_61,per_ocu_62,per_ocu_71,per_ocu_72,per_ocu_81,per_ocu_93,per_ocu_99,per_ocu_31-33,per_ocu_48-49
ageb,,,,,,,,,,,,,,,,,,,,,
0126,0.00,0.00,1.00,1.00,29.00,1008.00,0.00,11.00,11.00,4.00,...,9.96,20.13,6.07,2.00,4.87,2.84,20.00,19.30,8.22,34.10
0130,0.00,0.00,0.00,3.00,38.00,920.00,6.00,15.00,13.00,39.00,...,2.00,8.21,2.75,2.00,3.61,2.23,35.00,2.50,10.97,10.00
0198,0.00,0.00,0.00,1.00,16.00,155.00,1.00,0.00,7.00,9.00,...,2.50,11.18,4.53,3.20,3.03,2.67,8.00,2.00,8.02,20.00
0338,0.00,0.00,0.00,0.00,6.00,87.00,2.00,0.00,5.00,1.00,...,2.00,64.00,2.00,2.00,3.68,2.00,2.00,2.00,40.25,0.00
0361,0.00,0.00,0.00,2.00,12.00,81.00,5.00,4.00,12.00,37.00,...,3.88,7.65,3.89,5.00,11.52,4.77,16.00,5.60,4.78,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6572,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6619,0.00,0.00,0.00,0.00,0.00,7.00,0.00,0.00,0.00,0.00,...,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00
6623,0.00,0.00,0.00,0.00,1.00,20.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,2.00,0.00


In [18]:
AGEB_2010 = AGEB_2010.join(DEUNE_10_final, how='left')
AGEB_2010

,CODIGO,CVEGEO,GEOGRAFICO,FECHAACT,GEOMETRIA,INSTITUCIO,geometry,ENTIDAD,NOM_ENT,MUN,...,per_ocu_56,per_ocu_61,per_ocu_62,per_ocu_71,per_ocu_72,per_ocu_81,per_ocu_93,per_ocu_99,per_ocu_31-33,per_ocu_48-49
AGEB,,,,,,,,,,,,,,,,,,,,,
0126,LA_6052,2111400010126,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2898224.145 790876.493, 2898177.799 ...",21,Puebla,114,...,9.96,20.13,6.07,2.00,4.87,2.84,20.00,19.30,8.22,34.10
0130,LA_6052,2111400010130,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2899295.428 790736.219, 2899196.837 ...",21,Puebla,114,...,2.00,8.21,2.75,2.00,3.61,2.23,35.00,2.50,10.97,10.00
0198,LA_6052,2111400010198,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2894649.682 791049.840, 2894598.676 ...",21,Puebla,114,...,2.50,11.18,4.53,3.20,3.03,2.67,8.00,2.00,8.02,20.00
0338,LA_6052,2111400010338,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2903500.313 789608.302, 2903450.076 ...",21,Puebla,114,...,2.00,64.00,2.00,2.00,3.68,2.00,2.00,2.00,40.25,0.00
0361,LA_6052,2111400010361,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896010.210 788743.321, 2895949.550 ...",21,Puebla,114,...,3.88,7.65,3.89,5.00,11.52,4.77,16.00,5.60,4.78,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,LA_6052,2111400016676,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896630.713 779057.627, 2896652.213 ...",21,Puebla,114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6680,LA_6052,2111400016680,ÁREA GEOESTADÍSTICA BÁSICA,04/2010,ÁREA,INEGI,"POLYGON ((2893816.679 778660.123, 2893817.976 ...",21,Puebla,114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6695,LA_6052,2111403596695,ÁREA GEOESTADÍSTICA BÁSICA,05/2010,ÁREA,INEGI,"POLYGON ((2902665.816 781291.165, 2902472.591 ...",21,Puebla,114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
AGEB_2010.index.name = 'AGEB'
AGEB_2010

,CODIGO,CVEGEO,GEOGRAFICO,FECHAACT,GEOMETRIA,INSTITUCIO,geometry,ENTIDAD,NOM_ENT,MUN,...,per_ocu_56,per_ocu_61,per_ocu_62,per_ocu_71,per_ocu_72,per_ocu_81,per_ocu_93,per_ocu_99,per_ocu_31-33,per_ocu_48-49
AGEB,,,,,,,,,,,,,,,,,,,,,
0126,LA_6052,2111400010126,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2898224.145 790876.493, 2898177.799 ...",21,Puebla,114,...,9.96,20.13,6.07,2.00,4.87,2.84,20.00,19.30,8.22,34.10
0130,LA_6052,2111400010130,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2899295.428 790736.219, 2899196.837 ...",21,Puebla,114,...,2.00,8.21,2.75,2.00,3.61,2.23,35.00,2.50,10.97,10.00
0198,LA_6052,2111400010198,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2894649.682 791049.840, 2894598.676 ...",21,Puebla,114,...,2.50,11.18,4.53,3.20,3.03,2.67,8.00,2.00,8.02,20.00
0338,LA_6052,2111400010338,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2903500.313 789608.302, 2903450.076 ...",21,Puebla,114,...,2.00,64.00,2.00,2.00,3.68,2.00,2.00,2.00,40.25,0.00
0361,LA_6052,2111400010361,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896010.210 788743.321, 2895949.550 ...",21,Puebla,114,...,3.88,7.65,3.89,5.00,11.52,4.77,16.00,5.60,4.78,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,LA_6052,2111400016676,ÁREA GEOESTADÍSTICA BÁSICA,12/2007,ÁREA,INEGI,"POLYGON ((2896630.713 779057.627, 2896652.213 ...",21,Puebla,114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6680,LA_6052,2111400016680,ÁREA GEOESTADÍSTICA BÁSICA,04/2010,ÁREA,INEGI,"POLYGON ((2893816.679 778660.123, 2893817.976 ...",21,Puebla,114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6695,LA_6052,2111403596695,ÁREA GEOESTADÍSTICA BÁSICA,05/2010,ÁREA,INEGI,"POLYGON ((2902665.816 781291.165, 2902472.591 ...",21,Puebla,114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
AGEB_2010.to_csv("AGEB_10.csv", index=True)
AGEB_2010.to_file("/home/javier-rica-o/Escritorio/Ciencia de datos/Proyecto FInal/Datos/AGEB_10/AGEB_2010.shp", driver="ESRI Shapefile")

/tmp/ipykernel_7638/3589333073.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  AGEB_2010.to_file("/home/javier-rica-o/Escritorio/Ciencia de datos/Proyecto FInal/Datos/AGEB_10/AGEB_2010.shp", driver="ESRI Shapefile")


In [21]:
AGEB_20 = gpd.read_file("/home/javier-rica-o/Escritorio/Ciencia de datos/Proyecto FInal/Datos/AGEB_20/AGEB_2020.shp")
AGEB_10 = gpd.read_file("/home/javier-rica-o/Escritorio/Ciencia de datos/Proyecto FInal/Datos/AGEB_10/AGEB_2010.shp")

In [22]:
set(AGEB_10.columns)

{'AGEB',
 'CODIGO',
 'CVEGEO',
 'ENTIDAD',
 'FECHAACT',
 'GEOGRAFICO',
 'GEOMETRIA',
 'GRAPROES',
 'GRAPROES_F',
 'GRAPROES_M',
 'HOGJEF_F',
 'HOGJEF_M',
 'INSTITUCIO',
 'LOC',
 'MUN',
 'MZA',
 'NOM_ENT',
 'NOM_LOC',
 'NOM_MUN',
 'OCUPVIVPAR',
 'P12A14NOA',
 'P12A14NOAF',
 'P12A14NOAM',
 'P12YM_CASA',
 'P12YM_SEPA',
 'P12YM_SOLT',
 'P15A17A',
 'P15A17A_F',
 'P15A17A_M',
 'P15PRI_CO',
 'P15PRI_COF',
 'P15PRI_COM',
 'P15PRI_IN',
 'P15PRI_INF',
 'P15PRI_INM',
 'P15SEC_CO',
 'P15SEC_COF',
 'P15SEC_COM',
 'P15SEC_IN',
 'P15SEC_INF',
 'P15SEC_INM',
 'P15YM_AN',
 'P15YM_AN_F',
 'P15YM_AN_M',
 'P15YM_SE',
 'P15YM_SE_F',
 'P15YM_SE_M',
 'P18A24A',
 'P18A24A_F',
 'P18A24A_M',
 'P18YM_PB',
 'P18YM_PB_F',
 'P18YM_PB_M',
 'P3A5_NOA',
 'P3A5_NOA_F',
 'P3A5_NOA_M',
 'P3HLINHE',
 'P3HLINHE_F',
 'P3HLINHE_M',
 'P3HLI_HE',
 'P3HLI_HE_F',
 'P3HLI_HE_M',
 'P3YM_HLI',
 'P3YM_HLI_F',
 'P3YM_HLI_M',
 'P5_HLI',
 'P5_HLI_HE',
 'P5_HLI_NHE',
 'P6A11_NOA',
 'P6A11_NOAF',
 'P6A11_NOAM',
 'P8A14AN',
 'P8A14AN_F',


In [23]:
variantes=AGEB_20.columns
variantes

Index(['AGEB', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'ENTIDAD', 'NOM_ENT',
       'MUN', 'NOM_MUN', 'LOC',
       ...
       'per_ocu_61', 'per_ocu_62', 'per_ocu_71', 'per_ocu_72', 'per_ocu_81',
       'per_ocu_93', 'per_ocu_31', 'per_ocu_48', 'PNACE_%', 'geometry'],
      dtype='object', length=130)

In [24]:
AGEB_10 = AGEB_10[[col for col in AGEB_10.columns if col in variantes]]
AGEB_10

,AGEB,CVEGEO,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,MZA,POBTOT,...,per_ocu_56,per_ocu_61,per_ocu_62,per_ocu_71,per_ocu_72,per_ocu_81,per_ocu_93,per_ocu_31,per_ocu_48,geometry
0,0126,2111400010126,21,Puebla,114,Puebla,1,Total AGEB urbana,0,4819,...,9.96,20.13,6.07,2.00,4.87,2.84,20.00,8.22,34.10,"POLYGON ((2898224.145 790876.493, 2898177.799 ..."
1,0130,2111400010130,21,Puebla,114,Puebla,1,Total AGEB urbana,0,6423,...,2.00,8.21,2.75,2.00,3.61,2.23,35.00,10.97,10.00,"POLYGON ((2899295.428 790736.219, 2899196.837 ..."
2,0198,2111400010198,21,Puebla,114,Puebla,1,Total AGEB urbana,0,5256,...,2.50,11.18,4.53,3.20,3.03,2.67,8.00,8.02,20.00,"POLYGON ((2894649.682 791049.840, 2894598.676 ..."
3,0338,2111400010338,21,Puebla,114,Puebla,1,Total AGEB urbana,0,3406,...,2.00,64.00,2.00,2.00,3.68,2.00,2.00,40.25,0.00,"POLYGON ((2903500.313 789608.302, 2903450.076 ..."
4,0361,2111400010361,21,Puebla,114,Puebla,1,Total AGEB urbana,0,1789,...,3.88,7.65,3.89,5.00,11.52,4.77,16.00,4.78,0.00,"POLYGON ((2896010.210 788743.321, 2895949.550 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,6676,2111400016676,21,Puebla,114,Puebla,1,Total AGEB urbana,0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((2896630.713 779057.627, 2896652.213 ..."
486,6680,2111400016680,21,Puebla,114,Puebla,1,Total AGEB urbana,0,935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((2893816.679 778660.123, 2893817.976 ..."
487,6695,2111403596695,21,Puebla,114,Puebla,359,Total AGEB urbana,0,145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((2902665.816 781291.165, 2902472.591 ..."
488,6712,2111404886712,21,Puebla,114,Puebla,488,Total AGEB urbana,0,2850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((2904318.932 783248.086, 2904250.889 ..."


In [25]:
faltantes=[set(AGEB_20.columns)-set(AGEB_10.columns)]
faltantes

[{'CVE_ENT',
  'CVE_LOC',
  'CVE_MUN',
  'PNACE_%',
  'VIVPARH_CV',
  'VPH_AEASP',
  'VPH_BICI',
  'VPH_CISTER',
  'VPH_CVJ',
  'VPH_DSADMA',
  'VPH_HMICRO',
  'VPH_MOTO',
  'VPH_NDACMM',
  'VPH_NDEAED',
  'VPH_SINCIN',
  'VPH_SINLTC',
  'VPH_SINRTV',
  'VPH_SPMVPI',
  'VPH_STVP',
  'VPH_TINACO'}]

In [26]:
AGEB_10 = AGEB_10[AGEB_10['AGEB'] != '3008']
AGEB_10 = AGEB_10[AGEB_10['AGEB'] != '3169']
AGEB_10.iloc[:, 9:-1] = AGEB_10.iloc[:, 9:-1].apply(pd.to_numeric, errors='coerce')

In [28]:
AGEB_10 = AGEB_10.dropna(subset=['PNACOE'])
AGEB_10 = AGEB_10.dropna(subset=['grupo_11'])
#AGEB_10 = AGEB_10.drop('VPH_S_ELEC', axis=1)
#AGEB_10 = AGEB_10.drop('VPH_NODREN', axis=1)
#AGEB_10 = AGEB_10.drop('VPH_SNBIEN', axis=1)

columnas_a_imputar = [
    'POBMAS','POBFEM','PNACOE_M','PNACOE_F','PRESOE','PRESOE_M',
    'PRESOE_F','PDESOCUP','PDESOCUP_F','PDESOCUP_M','TVIVPARHAB',
    'VIVPAR_DES','VIVPAR_DES','VIVPAR_UT','VPH_1DOR',
    'VPH_1CUART','VPH_2CUART','VPH_AGUADV','VPH_C_SERV',
    'VPH_LAVAD','VPH_PC','VPH_TELEF','VPH_INTER','VPH_AUTOM','PE_INAC_M'
]

for col in columnas_a_imputar:
    AGEB_10[col] = AGEB_10[col].fillna(AGEB_10[col].mean())

In [29]:
nan_counts = AGEB_10.isna().sum()

# Convertir a DataFrame para mostrar de manera más ordenada
nan_df = pd.DataFrame(nan_counts).reset_index()
nan_df.columns = ['Column', 'NaN_Count']
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 50)
pd.set_option('display.max_colwidth', 50)
# Mostrar el DataFrame resultanten
nan_df

,Column,NaN_Count
0,AGEB,0
1,CVEGEO,0
2,ENTIDAD,0
3,NOM_ENT,0
4,MUN,0
5,NOM_MUN,0
6,LOC,0
7,NOM_LOC,0
8,MZA,0
9,POBTOT,0


In [30]:
AGEB_10

,AGEB,CVEGEO,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,MZA,POBTOT,POBMAS,POBFEM,PNACENT,PNACENT_M,PNACENT_F,PNACOE,PNACOE_M,PNACOE_F,PRES,PRES_M,PRES_F,PRESOE,PRESOE_M,PRESOE_F,GRAPROES,GRAPROES_M,GRAPROES_F,PEA,PEA_M,PEA_F,PE_INAC,PE_INAC_M,PE_INAC_F,POCUPADA,POCUPADA_M,POCUPADA_F,PDESOCUP,PDESOCUP_M,PDESOCUP_F,VIVTOT,TVIVHAB,TVIVPAR,VIVPAR_HAB,TVIVPARHAB,VIVPAR_DES,VIVPAR_UT,OCUPVIVPAR,PROM_OCUP,PRO_OCUP_C,VPH_PISODT,VPH_1DOR,VPH_2YMASD,VPH_1CUART,VPH_2CUART,VPH_3YMASC,VPH_C_ELEC,VPH_AGUADV,VPH_EXCSA,VPH_DRENAJ,VPH_C_SERV,VPH_RADIO,VPH_TV,VPH_REFRI,VPH_LAVAD,VPH_AUTOM,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,grupo_11,grupo_21,grupo_22,grupo_23,grupo_43,grupo_46,grupo_51,grupo_52,grupo_53,grupo_54,grupo_55,grupo_56,grupo_61,grupo_62,grupo_71,grupo_72,grupo_81,grupo_93,grupo_31-3,grupo_48-4,per_ocu_11,per_ocu_21,per_ocu_22,per_ocu_23,per_ocu_43,per_ocu_46,per_ocu_51,per_ocu_52,per_ocu_53,per_ocu_54,per_ocu_55,per_ocu_56,per_ocu_61,per_ocu_62,per_ocu_71,per_ocu_72,per_ocu_81,per_ocu_93,per_ocu_31,per_ocu_48,geometry
0,0126,2111400010126,21,Puebla,114,Puebla,1,Total AGEB urbana,0,4819,2275.00,2544.00,3917,1846,2071,682.00,321.00,361.00,4134,1939,2195,153.00,71.00,82.00,11.18,11.41,10.99,2132,1234.00,898.00,1642.00,514.00,1128.00,2055,1185.00,870.00,77.00,49.00,28.00,1571,1325,1516,1270,1324.00,184.00,62.00,4655,3.67,0.86,1222.00,341.00,917.00,102.00,113.00,1043.00,1255,1236.00,1256,1253.00,1228.00,1136,1232,1108.00,913.00,619.00,601.00,845.00,927.00,491.00,0.00,0.00,1.00,1.00,29.00,1008.00,0.00,11.00,11.00,4.00,0.00,24.00,38.00,28.00,8.00,167.00,166.00,4.00,79.00,28.00,0.00,0.00,8.00,2.00,5.38,3.08,0.00,3.64,2.00,2.00,0.00,9.96,20.13,6.07,2.00,4.87,2.84,20.00,8.22,34.10,"POLYGON ((2898224.145 790876.493, 2898177.799 ..."
1,0130,2111400010130,21,Puebla,114,Puebla,1,Total AGEB urbana,0,6423,3057.00,3366.00,5393,2591,2802,879.00,392.00,487.00,5495,2616,2879,220.00,100.00,120.00,10.43,10.76,10.14,2847,1717.00,1130.00,2123.00,611.00,1512.00,2727,1636.00,1091.00,120.00,81.00,39.00,2166,1769,2125,1728,1769.00,318.00,79.00,6300,3.65,0.92,1708.00,673.00,1048.00,98.00,244.00,1378.00,1722,1701.00,1718,1718.00,1698.00,1538,1680,1500.00,1157.00,643.00,635.00,908.00,1316.00,422.00,0.00,0.00,0.00,3.00,38.00,920.00,6.00,15.00,13.00,39.00,1.00,24.00,47.00,72.00,6.00,132.00,235.00,5.00,115.00,9.00,0.00,0.00,0.00,28.33,8.21,4.04,2.00,6.80,4.77,3.38,2.00,2.00,8.21,2.75,2.00,3.61,2.23,35.00,10.97,10.00,"POLYGON ((2899295.428 790736.219, 2899196.837 ..."
2,0198,2111400010198,21,Puebla,114,Puebla,1,Total AGEB urbana,0,5256,2458.00,2798.00,4444,2096,2348,685.00,294.00,391.00,4599,2121,2478,166.00,73.00,93.00,10.56,10.82,10.34,2376,1344.00,1032.00,1806.00,546.00,1260.00,2241,1255.00,986.00,135.00,89.00,46.00,1643,1443,1610,1410,1442.00,166.00,34.00,5150,3.65,0.88,1401.00,462.00,946.00,46.00,175.00,1185.00,1405,1339.00,1405,1406.00,1333.00,1276,1373,1236.00,969.00,518.00,570.00,875.00,1014.00,415.00,0.00,0.00,0.00,1.00,16.00,155.00,1.00,0.00,7.00,9.00,0.00,12.00,17.00,19.00,5.00,29.00,90.00,4.00,56.00,1.00,0.00,0.00,0.00,8.00,20.19,2.75,20.00,0.00,2.00,2.00,0.00,2.50,11.18,4.53,3.20,3.03,2.67,8.00,8.02,20.00,"POLYGON ((2894649.682 791049.840, 2894598.676 ..."
3,0338,2111400010338,21,Puebla,114,Puebla,1,Total AGEB urbana,0,3406,1632.00,1774.00,3019,1448,1571,236.00,109.00,127.00,2972,1412,1560,24.00,10.00,14.00,10.04,10.27,9.83,1414,854.00,560.00,1199.00,376.00,823.00,1338,801.00,537.00,76.00,53.00,23.00,1048,924,1002,878,924.00,101.00,23.00,3268,3.72,0.95,851.00,315.00,543.00,42.00,129.00,684.00,862,846.00,857,856.00,843.00,768,845,723.00,617.00,310.00,269.00,541.00,593.00,168.00,0.00,0.00,0.00,0.00,6.00,87.00,2.00,0.00,5.00,1.00,0.00,10.00,5.00,8.00,4.00,25.00,31.00,1.00,31.00,0.00,0.00,0.00,0.00,0.00,3.00,2.84,2.00,0.00,2.00,2.00,0.00,2.00,64.00,2.00,2.00,3.68,2.00,2.00,40.25,0.00,"POLYGON ((2903500.313 789608.302, 2903450.076 ..."
4,0361,2111400010361,21,Puebla,114,Puebla,1,Total AGEB urbana,0,1789,845.00,944.00,1228,575,653,317.00,147.00,170.00,1381,634,747,

In [31]:
den = AGEB_10['PNACENT'] + AGEB_10['PNACOE']
AGEB_10['PNACE_%'] =  AGEB_10['PNACENT'] / den

In [32]:
AGEB_10.to_file("/home/javier-rica-o/Escritorio/Ciencia de datos/Proyecto FInal/Datos/AGEB_10/AGEB_2010.shp", driver="ESRI Shapefile")